In [2]:
!pip install cplex
import cplex
print(cplex.__version__)


!pip install docplex==2.27.239
!pip install openpyxl

22.1.1.0
     ---------------------------------------- 0.0/635.6 kB ? eta -:--:--
     ----- --------------------------------- 92.2/635.6 kB 2.6 MB/s eta 0:00:01
     ------------ ------------------------- 204.8/635.6 kB 2.5 MB/s eta 0:00:01
     ---------------------- --------------- 368.6/635.6 kB 2.6 MB/s eta 0:00:01
     ----------------------------- -------- 501.8/635.6 kB 2.9 MB/s eta 0:00:01
     -------------------------------------  634.9/635.6 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 635.6/635.6 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  

In [7]:
# The module 'cplex' is not installed in the current environment. Let's switch to 'docplex', which is more commonly used for optimization modeling.

import pandas as pd
# from docplex.mp.model import Model
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta, date, time
import math
import cplex
import json

import os
os.environ['ILOG_LICENSE_FILE'] = 'D:\Cplex\python'

# Read data

In [6]:
d_path = '2024-04-25_OSRM_Truck_Distanzen+Fahrtzeiten_PZ_x_PZ.xlsx'    # Excel that contains the distance and time between service centers
sb_path = '2024-04-25_OR Praktikum_RWTH Aachen_WBeh_Aufträge.xlsx'  # Excel that contains the information of each Swapbodies and where it needs to go 
i_path = '2024-04-25_OR Praktikum_RWTH Aachen_Inputs.xlsx'       # Excel that contains the sorting capacity of each service center

# Read data from the Excel files into pandas DataFrames
df_times = pd.read_excel(d_path)
df_swapbodies = pd.read_excel(sb_path)
df_input = pd.read_excel(i_path, header=1, usecols=lambda x: 'Unnamed' not in x).dropna(axis='rows')



In [8]:
# Raname columns of excel files , simulating macros
class Rename:
    SORTING_FACILITY = "Sortierleistung [Sdg je h]"
    SWAPBODY = "id"
    PZ = 'PZA_GNR'
    PZA = "Origin_ID"
    PZE = "Destination_ID"
    QUANTITY = "Sendungsmenge"
    TIME = "OSRM_time [sek]"
    PICK_UP_TIME = "geplantes_beladeende"
    SORTING_DOOR = "Entladeleistung je Entladeband (Tor) je Stunde"
    SHIFT_END = "Auflegeende (=Sortierschluss/ PZE Sorter Cutoff)"
    SHIFT_BEGIN = "Schichtbeginn"

df_times[Rename.TIME] = df_times[Rename.TIME]/3600
df_swapbodies.rename(columns = {"quelle_agnr":"Origin_ID"}, inplace = True)
df_swapbodies.rename(columns = {"senke_agnr":"Destination_ID"}, inplace = True)

# Convert the DateTime column to datetime format
df_swapbodies[Rename.PICK_UP_TIME] = pd.to_datetime(df_swapbodies[Rename.PICK_UP_TIME], format='%d/%m/%Y %H:%M').dt.time



In [16]:
df_swapbodies[Rename.PICK_UP_TIME]

0      13:00:00
1      17:30:00
2      18:00:00
3      20:15:00
4      12:00:00
         ...   
195    20:15:00
196    18:00:00
197    20:15:00
198    17:50:00
199    18:00:00
Name: geplantes_beladeende, Length: 200, dtype: object

# Amount of data loaded into the model

In [9]:
df_times = df_times.head(200)
df_swapbodies = df_swapbodies.head(200)
df_input = df_input.head(200)

# Time Windows // Trying to apply the time window as drawn in the diagram

In [10]:
from datetime import time

def add_float_to_time(time_obj, float_hours):
    """
    Add a float number of hours to a given time object.

    Args:
    - time_obj (datetime.time): The original time object to add hours to.
    - float_hours (float): The number of hours to add, as a float.

    Returns:
    - datetime.time: A new time object representing the updated time after adding hours.
    """

    # Convert time_obj to the number of seconds since midnight
    total_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    
    # Convert float_hours to seconds
    added_seconds = round(float_hours * 3600)
    
    # Add the added_seconds to total_seconds
    new_total_seconds = total_seconds + added_seconds
    
    # Calculate the new hours, minutes, and seconds
    new_hours = (new_total_seconds // 3600) % 24
    new_minutes = (new_total_seconds % 3600) // 60
    new_seconds = new_total_seconds % 60
    
    # Create a new time object
    new_time_obj = time(new_hours, new_minutes, new_seconds)
    
    return new_time_obj


# E + 1 Function

In [11]:
def calculate_hour_range(start_time, end_time):
    """
    Calculate the number of hours between two time points.

    Args:
        start_time (datetime.time): Start time.
        end_time (datetime.time): End time.

    Returns:
        float: Number of hours between start_time and end_time.
    """
    
    # Convert datetime.time to datetime.datetime to facilitate subtraction
    start_datetime = datetime.combine(date.today(), start_time)
    end_datetime = datetime.combine(date.today(), end_time)
    
    # Calculate timedelta
    if end_datetime >= start_datetime:
        # Calculate hours when end time is on the same day
        hour_range = (end_datetime - start_datetime).total_seconds() / 3600.0
    else:
        # Handle case where end time is on the next day (assuming it's a 24-hour cycle)
        hour_range = (timedelta(days=1) - (start_datetime - end_datetime)).total_seconds() / 3600.0
    
    return hour_range


In [12]:

import json
import math

def eplus(df_swapbodies, df_times, df_input):
    """
    Process swap body data to determine which swap bodies and in which trajectory achieve E + 1.

    Args:
        df_swapbodies (DataFrame): DataFrame containing swap body information.
        df_times (DataFrame): DataFrame containing time information.
        df_input (DataFrame): DataFrame containing input data.

    Returns:
        None
    """
    
    # Initialize the nested dictionary without specifying values
    eplus_output_dict = {}

    # Loop through each swap body in df_swapbodies
    for i in range(len(df_swapbodies)):
        swapbody_id = str(df_swapbodies.loc[i, Rename.SWAPBODY])  # Convert to string if necessary

        # Calculate the time when a swap body is ready (pick up) at PZA
        pze_value = df_swapbodies.loc[i, Rename.PZE]
        
        # Check if PZE value exists in input PZ data
        if pze_value in df_input[Rename.PZ].values:
            search_pz = pze_value
            pze_sb = df_swapbodies[df_swapbodies[Rename.PZE] == search_pz]

            if not pze_sb.empty:
                pick_up_time = pze_sb.iloc[0][Rename.PICK_UP_TIME]  # Get pick-up time from swap body data

                # Calculate arrival time of swap body at PZE
                origin_id = df_swapbodies.loc[i, Rename.PZA]
                destination_id = df_swapbodies.loc[i, Rename.PZE]

                # Find matching time data for origin and destination IDs
                match_pz_d = df_times[(df_times[Rename.PZA] == origin_id) & (df_times[Rename.PZE] == destination_id)]

                if not match_pz_d.empty:
                    match_pz_time = match_pz_d.iloc[0][Rename.TIME]  # Get travel time between PZA and PZE

                    arr_time = add_float_to_time(pick_up_time, match_pz_time)

                    match_pz_inp = df_input[df_input[Rename.PZ] == destination_id]

                    if not match_pz_inp.empty:
                        match_pz_shift_begin = match_pz_inp.iloc[0][Rename.SHIFT_BEGIN]
                        match_pz_shift_end = match_pz_inp.iloc[0][Rename.SHIFT_END]

                        # Adjust arrival time if it's before shift begin
                        if arr_time <= match_pz_shift_begin:
                            arr_time = match_pz_shift_begin

                        # Calculate available hours within shift time
                        hours_avail = calculate_hour_range(arr_time, match_pz_shift_end)

                        match_pz_sb = df_swapbodies[(df_swapbodies[Rename.PZA] == origin_id) & (df_swapbodies[Rename.PZE] == destination_id)]

                        if not match_pz_sb.empty:
                            match_pz_quantity = match_pz_sb.iloc[0][Rename.QUANTITY]
                            match_pz_sorting = match_pz_inp.iloc[0][Rename.SORTING_DOOR]

                            # Calculate sorting ratio based on quantity and sorting capacity
                            sorting_ratio = math.ceil(match_pz_quantity / match_pz_sorting)

                            # Check if enough hours available for sorting
                            hours_avail = hours_avail - sorting_ratio
                            if hours_avail  >= 0:
                                eplus_output_dict[swapbody_id] = {
                                    "PZA": origin_id,
                                    "PZE": destination_id,
                                    "Arrival Time": arr_time.strftime('%H:%M'),
                                    "Slack Time (hrs)": round(hours_avail, 2) 
                                }
                            else:
                                print("Not enough hours available for sorting")
                        else:
                            print("No matching swap body found for sorting calculation")
                    else:
                        print("No input data found for destination PZ")
                else:
                    print("No matching time data found for origin and destination IDs")
            else:
                print("No swap body found for PZE:", pze_value)
        else:
            print("PZE value not found in input PZ data")

    # Write output dictionary to JSON file
    file_path = 'eplusone.json'
    with open(file_path, 'w') as file:
        json.dump(eplus_output_dict, file, indent=4)


In [13]:
#Mute CPLEX logs
def muteCplexLog(model):
    model.set_log_stream(None)
    model.set_error_stream(None)
    model.set_warning_stream(None)
    model.set_results_stream(None)

In [14]:
def solve_transportation_problem(model, df_times, df_swapbodies, df_input, time_weight=1.0, quantity_weight=1.0):
    """
    Solves a transportation problem using CPLEX.

    Args:
    - model: CPLEX model instance
    - df_times: DataFrame containing time data
    - df_swapbodies: DataFrame containing swap body data
    - df_input: DataFrame containing input data
    - time_weight: Weight for time component in objective function
    - quantity_weight: Weight for quantity component in objective function
    """

    # Extract necessary data from the dataframes
    swapbodies = df_swapbodies[Rename.SWAPBODY].tolist()
    pzas = df_swapbodies[Rename.PZA].tolist()
    pzes = df_swapbodies[Rename.PZE].tolist()
    quantities = df_swapbodies[Rename.QUANTITY].tolist()
    
    # Convert Time from seconds to hours
    times = df_times.set_index([Rename.PZA, Rename.PZE])[Rename.TIME]   # Assuming TIME is in seconds, convert to hours
    times = times.to_dict()
    
    num_swapbodies = len(swapbodies)
    
    # Objective function: combine time and quantity
    obj = []
    x = []
    for i in range(num_swapbodies):
        pza_coord = pzas[i]
        pze_coord = pzes[i]
        time_component = 0
        if (pza_coord, pze_coord) in times:
            time = times[(pza_coord, pze_coord)]
            time_component = time_weight * time
        quantity_component = quantity_weight * quantities[i]
        total_component = time_component + quantity_component
        x.append(model.variables.add(obj=[total_component], lb=[0], ub=[1], types=['B'], names=[f'x_{i}'])[0])
    
    model.objective.set_sense(model.objective.sense.minimize)
    
    # Constraints: ensure each ID is moved from its PZA to PZE
    for i in range(num_swapbodies):
        model.linear_constraints.add(
            lin_expr=[cplex.SparsePair([x[i]], [1])],
            senses=['E'], rhs=[1], names=[f'move_{i}']
        )
    
    # Group quantities and sorting capacities by facility
    facility_quantities = df_swapbodies.groupby(Rename.PZE)[Rename.QUANTITY].sum().to_dict()
    facility_sorting = df_input.groupby(Rename.PZ)[Rename.SORTING_FACILITY].sum().to_dict()
    
    # Constraint: Total quantities of swap bodies at each facility <= Facility's sorting capacity
    for facility_name, total_quantity in facility_quantities.items():
        if facility_name in facility_sorting:
            facility_indices = [k for k in range(num_swapbodies) if pzes[k] == facility_name]
            if facility_indices:
                model.linear_constraints.add(
                    lin_expr=[cplex.SparsePair([x[i] for i in facility_indices], [quantities[i] for i in facility_indices])],
                    senses=['L'], rhs=[facility_sorting[facility_name]], names=[f'quantity_sorting_facility_{facility_name}']
                )
        else:
            print(f"Warning: Facility {facility_name} not found in facility_sorting")
    
    # Constraint: Max 2 swapbodies per truck
    for l in range(num_swapbodies):
        model.linear_constraints.add(
            lin_expr=[cplex.SparsePair([x[l]], [1])],
            senses=['L'], rhs=[2], names=[f'max_ids_per_truck_{l}']
        )

    # Mute CPLEX log for the transportation problem solving part
    muteCplexLog(model)
    
    # Solve the model
    model.solve()
    
    # Call eplus function to process additional data
    eplus(df_swapbodies, df_times, df_input)
    
    # Print solution
    if model.solution.is_primal_feasible():
        print('Objective Value:', model.solution.get_objective_value())
        # Print solution details if needed
        # for k in range(num_swapbodies):
        #     if model.solution.get_values(x[k]) > 0.9:
        #         print(f'Swap-body {swapbodies[k]} moved from PZA {pzas[k]} to PZE {pzes[k]}')
    else:
        print('No feasible solution found.')

# Running Instances

In [15]:
#Init model and mute CPLEX output
model = cplex.Cplex()
muteCplexLog(model)


solve_transportation_problem(model, df_times, df_swapbodies, df_input, time_weight=1, quantity_weight=1)

Objective Value: 203207.27511111108


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=191c9c86-2871-4134-855d-0821a157bdce' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>